<a href="https://colab.research.google.com/github/venkatacrc/Notes/blob/master/ML_GCP/ProductionML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Production ML Systems
Source: [Production Machine Learning Systems](https://www.coursera.org/learn/gcp-production-ml-systems/)

1. Architecting Production ML Systems
1. Ingesting data for Cloud-based analytics and ML
1. Designing Adaptable ML systems
1. Designing High Performance ML Systems
1. Hybrid ML Systems

###Architecting Production ML Systems
1. What's in a Production ML System
  * Data Collection
  * Data Verification
  * Machine Resource Management
  * Feature Extraction
  * Process Management Tools
  * Configuration
  * Monitoring
  * Analysis Tools
  * Serving Infrastructure
  * ML Code
1. Training Design Decisions
1. Serving Design Decisions
1. Serving on CMLE (scalability)
1. Designing an Architecture from Scratch

Reuse generic software frameworks
* TensorFlow
* TF Serving
* Apache Spark
* Apache Beam
Use managed services
* Cloud Dataproc
* Cloud Dataflow
* Cloud ML Engine







##The Components of an ML System

###Data Ingestion

**Streaming Data Ingestion Pipeline Architecture**

$
\left.\begin{array}{ccc}
{Applications} \\
{Devices} \\
{Databases}
\end{array}\right\} \rightarrow
\left.\begin{array}{ccc}
{(Ingest)}\\
{Cloud Pub/Sub}
\end{array}\right\} \rightarrow
\left.\begin{array}{ccc}
{(Process)}\\
{Cloud Dataflow}
\end{array}\right\} \rightarrow \left\{ 
\begin{array}{ccc}
{(Analyze)}\\
{Data Studio | Third Party Tools}\\
{\uparrow} \\
{Cloud BigQuery} \mapsto {Data Warehouse}\\ 
{Cloud MLE} \mapsto {Predictive Analytics}\\
{Cloud BigTable} \mapsto {Caching \& Serving} 
\end{array}\right.$